In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset,TensorDataset
from autoencoder import Autoencoder
from cifar_autoencoder import Cifar_Autoencoder
import torchvision
from model2 import classification_model
from model import cifar_classification_model
import copy
import partition
from pca import PCADigitReducer
from autoencoder import reduce_dimensions
from training import train,test,train_resnet,test_resnet
from federated_learning import distribute_global_model, federated_averaging
from torchvision.models import resnet18
from torchvision.models import mobilenet_v2
from model3 import MobileNetV2

In [6]:
model = resnet18(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 10)

c:\Users\micha\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\micha\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
model2 = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

Using cache found in C:\Users\micha/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\micha\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
model3 = MobileNetV2()

In [15]:
# Predefined stuff

n_epochs = 8
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10
num_clusters = 2

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [10]:
cifar10_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # https://pytorch.org/hub/pytorch_vision_resnet/
])

cifar10_train_loader = DataLoader(
    datasets.CIFAR10('/files/', train=True, download=True, transform=cifar10_transform),
    batch_size=batch_size_train, shuffle=True
)

cifar10_test_loader = DataLoader(
    datasets.CIFAR10('/files/', train=False, download=True, transform=cifar10_transform),
    batch_size=batch_size_test, shuffle=True
)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
class CustomTensorDataset(TensorDataset):
    def __init__(self, *tensors):
        super().__init__(*tensors)
        self.data = tensors[0]
        self.targets = tensors[1]

In [12]:
train_loader_pca = copy.copy(cifar10_train_loader)
test_loader_pca = copy.copy(cifar10_test_loader)

train_loader_auto = copy.copy(cifar10_train_loader)
test_loader_auto = copy.copy(cifar10_test_loader)

In [7]:
train_data = []
train_labels = []
for data, labels in train_loader_pca:  # Use your CIFAR-10 DataLoader here
    train_data.append(data.view(data.size(0), -1))  # Flatten images
    train_labels.append(labels)
train_data = torch.cat(train_data, dim=0)  # Combine all batches
train_labels = torch.cat(train_labels, dim=0)

# Convert to numpy for PCA
train_data_np = train_data.numpy()

# Perform PCA
n_components = 100  # Set the desired number of components
pca = PCADigitReducer(n_components)
train_data_reduced = pca.fit_transform(train_data_np)  # Reduce dimensions

# Reconstruct the dataset from the reduced dimensions
train_data_reconstructed_np = pca.inverse_transform(train_data_reduced) 
train_data_reconstructed = torch.tensor(train_data_reconstructed_np, dtype=torch.float32)

# Reshape the reconstructed data back into the original image dimensions
train_data_reconstructed = train_data_reconstructed.view(-1, 3, 32, 32)

# Normalize the reconstructed dataset (use CIFAR-10 mean and std)
train_data_reconstructed = (train_data_reconstructed - torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)) / \
                           torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)

# Create a new DataLoader for the reconstructed data
batch_size_train = cifar10_train_loader.batch_size
train_dataset_pca = CustomTensorDataset(train_data_reconstructed, train_labels)
train_loader_reduced_pca = DataLoader(train_dataset_pca, batch_size=batch_size_train, shuffle=True)

In [8]:
latent_dim = 100  # Adjust latent dimension as needed
autoencoder = Cifar_Autoencoder(latent_dim=latent_dim)
auto_criterion = nn.MSELoss()
auto_optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)
auto_num_epochs = 5

for epoch in range(auto_num_epochs):
    for images, _ in cifar10_train_loader:  # Use your CIFAR-10 DataLoader here
        auto_optimizer.zero_grad()
        
        # Forward pass
        reconstructed = autoencoder(images)
        
        # Compute reconstruction loss
        loss = auto_criterion(reconstructed, images)
        
        # Backward pass and optimization
        loss.backward()
        auto_optimizer.step()
        
    print(f"Epoch [{epoch+1}/{auto_num_epochs}], Loss: {loss.item()}")


Epoch [1/5], Loss: 0.7032589912414551
Epoch [2/5], Loss: 0.7096912860870361
Epoch [3/5], Loss: 0.6233389377593994
Epoch [4/5], Loss: 0.5864422917366028
Epoch [5/5], Loss: 0.6471503376960754


In [9]:
autoencoder.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
latent_features, labels = reduce_dimensions(train_loader_auto, autoencoder.encoder, device)
latent_features = latent_features.detach()

reconstructed_images = autoencoder.decoder(latent_features.to(device))  
reconstructed_images = reconstructed_images.view(-1, 3, 32, 32) # Reshape to [batch_size, channels, height, width]

reconstructed_dataset = CustomTensorDataset(reconstructed_images.cpu(), labels)  
reduced_train_loader_auto = DataLoader(reconstructed_dataset, batch_size=batch_size_train, shuffle=True)

In [10]:
# Now partition them into 4 clients for federated learning
# pca 4 clients
trainingset_pca = train_loader_reduced_pca.dataset
partitioned_data_pca = partition.balanced_dirichlet_partition(trainingset_pca, partitions_number=4, alpha=0.5)
# Check sizes

for i in partitioned_data_pca:
    print(len(partitioned_data_pca[i]))

18580
9043
14723
7654


In [11]:
pca_client_loaders = [
    DataLoader(Subset(trainingset_pca, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_pca.values()
]

In [12]:
# classic
trainingset = cifar10_train_loader.dataset
partitioned_data_classic = partition.balanced_dirichlet_partition(trainingset, partitions_number=4, alpha=0.5)

classic_client_loaders = [
    DataLoader(Subset(trainingset, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_classic.values()
]

In [13]:
# auto 4 clients
trainingset_auto = reduced_train_loader_auto.dataset
partitioned_data_auto = partition.balanced_dirichlet_partition(trainingset_auto, partitions_number=4, alpha=0.5)

auto_client_loaders = [
    DataLoader(Subset(trainingset_auto, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_auto.values()
]

In [14]:
# defining model for pca and autoencoder

global_model_pca = model
global_model_auto = model
gloabl_model_classic = model

num_clients = 4
# classic model
local_models_classic = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]
# pca models 
local_models_pca = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]
# autoencodere models
local_model_autoencoder = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]

In [18]:
testmodel = model

In [26]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train_resnet(epoch, model, cifar10_train_loader, optimizer, log_interval, train_losses, train_counter)
    test_losses = []
    test_resnet(model,test_loader_pca,test_losses)

Train Epoch: 1 [0/50000 (0%)]	Loss: 0.042231
Train Epoch: 1 [640/50000 (1%)]	Loss: 0.069573
Train Epoch: 1 [1280/50000 (3%)]	Loss: 0.011470
Train Epoch: 1 [1920/50000 (4%)]	Loss: 0.003030
Train Epoch: 1 [2560/50000 (5%)]	Loss: 0.007982
Train Epoch: 1 [3200/50000 (6%)]	Loss: 0.004587
Train Epoch: 1 [3840/50000 (8%)]	Loss: 0.007837
Train Epoch: 1 [4480/50000 (9%)]	Loss: 0.046453
Train Epoch: 1 [5120/50000 (10%)]	Loss: 0.001544
Train Epoch: 1 [5760/50000 (12%)]	Loss: 0.026832
Train Epoch: 1 [6400/50000 (13%)]	Loss: 0.073292
Train Epoch: 1 [7040/50000 (14%)]	Loss: 0.015311
Train Epoch: 1 [7680/50000 (15%)]	Loss: 0.002939
Train Epoch: 1 [8320/50000 (17%)]	Loss: 0.010287
Train Epoch: 1 [8960/50000 (18%)]	Loss: 0.013606
Train Epoch: 1 [9600/50000 (19%)]	Loss: 0.021687
Train Epoch: 1 [10240/50000 (20%)]	Loss: 0.007346
Train Epoch: 1 [10880/50000 (22%)]	Loss: 0.021030
Train Epoch: 1 [11520/50000 (23%)]	Loss: 0.007495
Train Epoch: 1 [12160/50000 (24%)]	Loss: 0.050750
Train Epoch: 1 [12800/50000 

KeyboardInterrupt: 

In [21]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train_resnet(epoch, model, cifar10_train_loader, optimizer, log_interval, train_losses, train_counter)
    test_losses = []
    test_resnet(testmodel,test_loader_pca,test_losses)

Train Epoch: 1 [0/50000 (0%)]	Loss: 0.050591
Train Epoch: 1 [640/50000 (1%)]	Loss: 0.108985
Train Epoch: 1 [1280/50000 (3%)]	Loss: 0.126027
Train Epoch: 1 [1920/50000 (4%)]	Loss: 0.117060
Train Epoch: 1 [2560/50000 (5%)]	Loss: 0.216151
Train Epoch: 1 [3200/50000 (6%)]	Loss: 0.053155
Train Epoch: 1 [3840/50000 (8%)]	Loss: 0.125457
Train Epoch: 1 [4480/50000 (9%)]	Loss: 0.098552
Train Epoch: 1 [5120/50000 (10%)]	Loss: 0.107360
Train Epoch: 1 [5760/50000 (12%)]	Loss: 0.071858
Train Epoch: 1 [6400/50000 (13%)]	Loss: 0.073997
Train Epoch: 1 [7040/50000 (14%)]	Loss: 0.215753
Train Epoch: 1 [7680/50000 (15%)]	Loss: 0.227913
Train Epoch: 1 [8320/50000 (17%)]	Loss: 0.097784
Train Epoch: 1 [8960/50000 (18%)]	Loss: 0.162740
Train Epoch: 1 [9600/50000 (19%)]	Loss: 0.174905
Train Epoch: 1 [10240/50000 (20%)]	Loss: 0.156967
Train Epoch: 1 [10880/50000 (22%)]	Loss: 0.151476
Train Epoch: 1 [11520/50000 (23%)]	Loss: 0.196193
Train Epoch: 1 [12160/50000 (24%)]	Loss: 0.174179
Train Epoch: 1 [12800/50000 

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses2 = []
train_counter2 = []

for epoch in range(1, n_epochs + 1):  
    train_resnet(epoch, model3, cifar10_train_loader, optimizer, log_interval, train_losses2, train_counter2)
    test_losses = []
    test_resnet(model3,test_loader_pca,test_losses)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.292453
Train Epoch: 1 [640/50000 (1%)]	Loss: 2.288550
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.327415
Train Epoch: 1 [1920/50000 (4%)]	Loss: 2.294276
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.319928
Train Epoch: 1 [3200/50000 (6%)]	Loss: 2.334350
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.333319
Train Epoch: 1 [4480/50000 (9%)]	Loss: 2.300523
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.361311
Train Epoch: 1 [5760/50000 (12%)]	Loss: 2.352684
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.300988
Train Epoch: 1 [7040/50000 (14%)]	Loss: 2.318786
Train Epoch: 1 [7680/50000 (15%)]	Loss: 2.326198
Train Epoch: 1 [8320/50000 (17%)]	Loss: 2.306401
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.318380
Train Epoch: 1 [9600/50000 (19%)]	Loss: 2.336225
Train Epoch: 1 [10240/50000 (20%)]	Loss: 2.326647
Train Epoch: 1 [10880/50000 (22%)]	Loss: 2.315620
Train Epoch: 1 [11520/50000 (23%)]	Loss: 2.314243
Train Epoch: 1 [12160/50000 (24%)]	Loss: 2.304264
Train Epoch: 1 [12800/50000 

TypeError: train_resnet() missing 4 required positional arguments: 'optimizer', 'log_interval', 'train_losses', and 'train_counter'

In [16]:
test_resnet(model3,test_loader_pca,test_losses)


Test set: Avg. loss: 2.3115, Accuracy: 970/10000 (10%)



In [17]:
rounds_pca = 4

for round_idx in range(rounds_pca):
    
    print(f"Round {round_idx + 1}/{rounds_pca}")

    local_weights_pca = []
    for client_idx, client_model in enumerate(local_models_pca):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []


        for epoch in range(1, n_epochs + 1):  
            train_resnet(epoch, client_model, pca_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_pca.append(client_weights)
        
    print(f"after training{local_models_pca}")
    global_weights_pca = federated_averaging(local_weights_pca)
    print(f"after fedaveraging{local_models_pca}")

    distribute_global_model(global_weights_pca,local_models_pca,single=False)

    distribute_global_model(global_weights_pca,global_model_pca,single=True)
    test_losses = []
    test_resnet(global_model_pca,test_loader_pca,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/18580 (0%)]	Loss: 1.272864
Train Epoch: 1 [640/18580 (3%)]	Loss: 0.395788
Train Epoch: 1 [1280/18580 (7%)]	Loss: 0.549133
Train Epoch: 1 [1920/18580 (10%)]	Loss: 0.684713
Train Epoch: 1 [2560/18580 (14%)]	Loss: 0.362403
Train Epoch: 1 [3200/18580 (17%)]	Loss: 0.319676
Train Epoch: 1 [3840/18580 (21%)]	Loss: 0.682662
Train Epoch: 1 [4480/18580 (24%)]	Loss: 0.539573
Train Epoch: 1 [5120/18580 (27%)]	Loss: 0.584719
Train Epoch: 1 [5760/18580 (31%)]	Loss: 0.337060
Train Epoch: 1 [6400/18580 (34%)]	Loss: 0.730592
Train Epoch: 1 [7040/18580 (38%)]	Loss: 0.541934
Train Epoch: 1 [7680/18580 (41%)]	Loss: 0.612196
Train Epoch: 1 [8320/18580 (45%)]	Loss: 0.350242
Train Epoch: 1 [8960/18580 (48%)]	Loss: 0.718655
Train Epoch: 1 [9600/18580 (52%)]	Loss: 0.459587
Train Epoch: 1 [10240/18580 (55%)]	Loss: 0.487602
Train Epoch: 1 [10880/18580 (58%)]	Loss: 0.585782
Train Epoch: 1 [11520/18580 (62%)]	Loss: 0.619536
Train Epoch: 1 [12160/18580 (65%)]	Loss: 0.45

In [ ]:
rounds_auto = 4

for round_idx in range(rounds_auto):
    print(f"Round {round_idx + 1}/{rounds_auto}")

    local_weights_auto = []
    for client_idx, client_model in enumerate(local_model_autoencoder):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)
        
        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, auto_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_auto.append(client_weights)
        
    global_weights_auto = federated_averaging(local_weights_auto)

    distribute_global_model(global_weights_auto,local_model_autoencoder,single=False)

    distribute_global_model(global_weights_auto, global_model_auto,single=True)
    test_losses = []
    test(global_model_auto,test_loader_auto,test_losses)

In [ ]:
rounds_classic = 4

for round_idx in range(rounds_classic):
    
    print(f"Round {round_idx + 1}/{rounds_classic}")

    local_weights_classic = []
    for client_idx, client_model in enumerate(local_models_classic):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train_resnet(epoch, client_model, classic_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_classic.append(client_weights)
        
    print(f"after training{local_models_classic}")
    global_weights_classic = federated_averaging(local_weights_classic)
    print(f"after fedaveraging{local_models_classic}")

    distribute_global_model(global_weights_classic,local_models_classic,single=False)

    distribute_global_model(global_weights_classic,gloabl_model_classic,single=True)
    test_losses = []
    test_resnet(gloabl_model_classic,cifar10_test_loader,test_losses)